# Route Helpers

> Helper utilities for resource monitoring route handlers

In [ ]:
#| default_exp utils.route_helpers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Callable, Dict, Any

## Card Container Wrapper

Wrap card content in a styled container for consistent presentation.

In [ ]:
#| export
def wrap_card_in_container(content, html_id, card_cls=None, bg_cls=None, shadow_cls=None, **kwargs):
    """
    Wrap card content in a Div container with standard styling.
    
    This consolidates the common pattern of wrapping monitoring cards
    in styled containers.
    
    Args:
        content: Card content to wrap
        html_id: HTML ID for the container
        card_cls: Card class (optional, can be provided via DaisyUI)
        bg_cls: Background class (optional, can be provided via DaisyUI)
        shadow_cls: Shadow class (optional, can be provided via Tailwind)
        **kwargs: Additional attributes for the Div
    
    Returns:
        Div: Wrapped card container
    """
    # This function provides a reusable pattern but remains flexible
    # Applications can provide their own styling classes
    
    try:
        from fasthtml.common import Div
        from cjm_fasthtml_tailwind.core.base import combine_classes
        
        # Build class list
        classes = []
        if card_cls:
            classes.append(card_cls)
        if bg_cls:
            classes.append(bg_cls)
        if shadow_cls:
            classes.append(shadow_cls)
        
        cls = combine_classes(*classes) if classes else None
        
        return Div(
            content,
            cls=cls,
            id=html_id,
            **kwargs
        )
    except ImportError:
        # Fallback if dependencies not available
        return content

Example usage with DaisyUI and Tailwind:

```python
from cjm_fasthtml_daisyui.components.data_display.card import card
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui
from cjm_fasthtml_tailwind.utilities.effects import shadow

wrapped = wrap_card_in_container(
    my_card_content,
    html_id="my-card",
    card_cls=card,
    bg_cls=bg_dui.base_100,
    shadow_cls=shadow.md
)
```

## OOB Card Update Helper

Create out-of-band (OOB) swap updates for cards in SSE streaming.

In [ ]:
#| export
def create_card_update(
    render_fn: Callable, 
    info: Dict[str, Any], 
    target_id: str, 
    swap_type: str = "outerHTML"
):
    """
    Create an OOB swap update for a card.
    
    This consolidates the pattern of creating OOB swaps for card updates
    in SSE streaming.
    
    Args:
        render_fn: Function to render the card
        info: Info dictionary to pass to render function
        target_id: Target HTML ID for the swap
        swap_type: Type of swap (default "outerHTML")
    
    Returns:
        OOB swap element
    """
    try:
        from cjm_fasthtml_sse.helpers import oob_swap
        
        return oob_swap(
            render_fn(info),
            target_id=target_id,
            swap_type=swap_type
        )
    except ImportError:
        # Fallback if SSE library not available
        return render_fn(info)

Example usage in SSE streaming:

```python
from cjm_fasthtml_sysmon.components.cards import render_cpu_card
from cjm_fasthtml_sysmon.monitors.cpu import get_cpu_info

# In your SSE streaming function
async def stream_updates():
    while True:
        cpu_info = get_cpu_info()
        
        # Create OOB update for CPU card
        update = create_card_update(
            render_cpu_card,
            cpu_info,
            "cpu-card-body"
        )
        
        yield sse_message(Div(update))
        await asyncio.sleep(2)
```

These helpers reduce boilerplate code when working with resource monitoring routes and SSE updates. They provide a consistent pattern while remaining flexible enough to work with different styling libraries and update strategies.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()